In [66]:
import numpy as np
import nibabel as nib
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
import tensorflow as tf

In [110]:
# Define constants
image_size = 128
num_slices = 155
slices_start_at = 0
num_channels = 4
num_classes = 4

In [70]:
# Load Images
def load_images(data_directory, sample_ids):
    '''
    Inputs:
        data_directory: The directory containing the dataset.
        sample_ids: A list of sample IDs to load.
    Steps:
        Loop Over Sample IDs: Iterate over each sample ID in the provided list.
        
        Load Image and Segmentation: For each sample ID, load the corresponding image and segmentation data from the dataset directory.
        
        Resize Images and Segmentations: Resize each image and segmentation slice to the specified image_size. The cv2.resize function is used for resizing.
        
        One-Hot Encode Segmentation: Convert the segmentation data into one-hot encoded format using tf.keras.utils.to_categorical.
        
        Append to Lists: Append the resized image slices (X) and their corresponding one-hot encoded segmentation slices (y) to separate lists.
        
        Convert to Arrays: Convert the lists X and y into numpy arrays.
        
        Normalization: Normalize the image data by dividing by the maximum pixel value, so that all pixel values will be on the same scale.
    Output:
        X: Numpy array containing the preprocessed image data.
        y: Numpy array containing the preprocessed segmentation data in one-hot encoded format.
    '''
    X = []
    y = []
    for sample_id in sample_ids:
        imgtr_path = os.path.join(data_directory, 'imagesTr', sample_id)
        lbltr_path = os.path.join(data_directory, 'labelsTr', sample_id)

        imgtr = nib.load(imgtr_path).get_fdata()
        lbltr = nib.load(lbltr_path).get_fdata()

        for j in range(num_slices):
            
            # Resize the j-th slice of the MRI image and append it to X
            X.append(cv2.resize(imgtr[:, :, j + slices_start_at], (image_size, image_size)))
            
            # Resize the j-th slice of the segmentation mask image and append its one-hot encoded version to y
            seg_slice = cv2.resize(lbltr[:, :, j + slices_start_at], (image_size, image_size))
            y.append(tf.keras.utils.to_categorical(seg_slice, num_classes))

    X = np.array(X) / np.max(X)
    y = np.array(y)
    return X, y

In [67]:
# Model Definition
def create_model(input_shape):
    '''
    Input Layer: The input shape is specified by input_shape.
    Encoder Path:
        Two convolutional layers (conv1) with 32 filters each are applied with ReLU activation, followed by max-pooling (pool1) to reduce spatial dimensions.
        Similar operations are performed in the next set of convolutional layers (conv2 and pool2), but with 64 filters.
        Another set of convolutional layers (conv3) with 128 filters further reduces the spatial dimensions.
    Decoder Path:
        Upsampling (UpSampling2D) is applied to the output of conv3, followed by a convolutional layer (up1) with 64 filters.
        The upsampled feature map is concatenated with the feature map from conv2 (merge1), followed by convolutional layers (conv4) with 64 filters.
        A similar operation is performed for the next upsampled feature map (up2) and the feature map from conv1 (merge2).
        Convolutional layers (conv5) with 32 filters are applied to refine the segmentation.
    Output Layer:
        The final layer (output) consists of a convolutional layer with num_classes filters (in this case, 4 for multi-class segmentation) and softmax activation to produce the segmentation map.
    Model Compilation:
        The model is instantiated with the defined inputs and outputs.
        The model is not compiled here. Compilation typically involves specifying an optimizer, loss function, and metrics, but this is deferred to the training phase.
    '''
    inputs = Input(input_shape)
    conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)

    up1 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=2)(conv3))
    merge1 = concatenate([conv2, up1], axis=3)

    conv4 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge1)
    conv4 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)

    up2 = Conv2D(32, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=2)(conv4))

    merge2 = concatenate([conv1, up2], axis=3)
    conv5 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge2)
    conv5 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)

    output = Conv2D(num_classes, 1, activation='softmax')(conv5)
    
    model = Model(inputs=inputs, outputs=output)
    return model

In [69]:
def train_model(data_directory, model, epochs, learning_rate=0.0001):
    '''
    Inputs:
        data_directory: The directory containing the training data.
        model: The model to be trained.
        epochs: The number of epochs for training.
        learning_rate: The learning rate for the Adam optimizer, with a default value of 0.0001.
    Steps followed in this function:
        Load Data: The function loads the training data using the load_data function, which loads the data from the specified directory.
        Preprocess Data: The training data is preprocessed using the load_images function, which prepares the input images (X_train) and their corresponding labels (y_train).
        Compile Model: The model is compiled using the Adam optimizer with the specified learning rate, categorical cross-entropy loss function, and accuracy metric.
        Model Training: The model is trained using the training data (X_train and y_train) for the specified number of epochs.
    '''
    train_samples, _ = load_data(data_directory)
    X_train, y_train = load_images(data_directory, train_samples)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs)

In [71]:
def predict_image(image_path, model):
    '''
    Inputs:
        image_path: The path to the input image.
        model: The trained model used for making predictions.
    Steps:
        Load Image: Load the input image data using nibabel.
        
        Resize Image: Iterate over the slices of the input image, resize each slice to the specified image_size, and append them to a list X.
        
        Normalization: Normalize the resized image data by dividing by the maximum pixel value.
        
        Add Channel Dimension: Add a channel dimension to the preprocessed image data to match the input shape expected by the model.
        
        Make Predictions: Use the trained model to predict segmentation masks for the preprocessed image data.
    Outputs:
        X: Numpy array containing the preprocessed image data.
        predictions: Predicted segmentation masks for the input image data.
    '''
    img_data = nib.load(image_path).get_fdata()

    X = []
    for j in range(img_data.shape[2]):
        X.append(cv2.resize(img_data[:, :, j], (image_size, image_size)))
    X = np.array(X) / np.max(X)
    X = np.expand_dims(X, axis=-1)  # Add channel dimension

    predictions = model.predict(X)
    return X, predictions


In [106]:
def visualize(input_image, output_image):
    '''
    Inputs:
        input_image: The input image data.
        output_image: The predicted segmentation mask.
    Steps:
        Plot Input Image: Plot the input image in the left subplot.
        The input image is accessed using indexing [65, :, :, 0, 0], assuming it's a 3D image with dimensions [depth, height, width, channels, samples]. Adjust the indexing according to the structure of your input data.
        The colormap 'gray' is used for grayscale images.
        Plot Output Image: Plot the output image (prediction) in the right subplot.
        The output image is accessed using indexing [65, :, :, 0], assuming it's a 3D segmentation mask with dimensions [depth, height, width, channels]. Adjust the indexing according to the structure of your output data.
        The colormap 'gray' is used for grayscale images.
    Display: Show the plotted images using plt.show().
    
    '''
    plt.figure(figsize=(10, 5))

    # Plot the input image
    plt.subplot(1, 2, 1)
    plt.imshow(input_image[65,:,:,0,0], cmap='gray')
    plt.title('Input Image')
    plt.axis('off')

    # Plot the output image
    plt.subplot(1, 2, 2)
    plt.imshow(output_image[65,:,:,0], cmap='gray')
    plt.title('Output Image')
    plt.axis('off')

    plt.show()

In [89]:
# Set the data directory
data_directory = "./Task01_BrainTumour"

# Define input shape
input_shape = (image_size, image_size, num_channels)

# Create the model
model = create_model(input_shape)

# Train the model
train_model(data_directory, model, epochs=10)

Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 12s 86ms/step - accuracy: 0.9352 - loss: 1.0510
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.9836 - loss: 0.0849
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.9845 - loss: 0.0593
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.9881 - loss: 0.0533
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.9889 - loss: 0.0375
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.9803 - loss: 0.0723
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.9890 - loss: 0.0389
Epoch 8/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.9886 - loss: 0.0447
Epoch 9/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.9548 - loss: 0.3307
Epoch 10/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.9900 - loss: 0.0373
